# 級数による円周率の数値計算

## 円周率 `pi`  

Juliaでは円周率は数学定数`pi`として実装されている．

In [1]:
pi           # 遅延評価．実際に計算されるときに適切な形で実体化される．

π = 3.1415926535897...

In [2]:
Float64(pi)    # `Float64` 型として実体化

3.141592653589793

この値は確かに16桁まで一致している（倍精度浮動小数点数では10進数で約16桁の表示が限界である）．
念のため，正確な値を明記しておく．
```
 3.14159_26535_89793
```


## Leibniz（ライプニッツ）の公式
交代級数に関する公式
$$
 1 - \dfrac{1}{3} + \dfrac{1}{5} - \dfrac{1}{7} + \cdots = \dfrac{\pi}{4}
$$
を **Leibnizの公式** と呼ぶ．これは収束が遅いので円周率の近似値計算には適さないが，試しに数値計算してみよう．

`for`ループを用いて無限級数を計算する素朴なコードは次のようになる．

In [3]:
n = 120
s = 0.0
for i in 0:n
   s += (-1)^i/(2i+1)
end
@show 4s

4s = 3.149856975293274


3.149856975293274

120+1項まで計算してやっと3桁まで一致する程度であり，やはり収束速度は非常に遅い．

上記コードでも数値計算としては問題ないが，Juliaでは内包表記と`sum()`を使えばもっとシンプルに書ける．

In [4]:
leibniz(n) = 4*sum((-1)^i/(2i+1) for i in 0:n)

leibniz (generic function with 1 method)

In [5]:
leibniz(120)

3.149856975293274

#### Note
`sum()`で総和を計算する場合は，`[...]`で囲わないほうが効率が良い．

In [6]:
@time sum(i for i in 1:100)
@time sum([i for i in 1:100])   # [...]で囲った場合

  0.000000 seconds
  0.000001 seconds (1 allocation: 896 bytes)


5050

どちらも同じ値を返すが，前者は`sum()`内の`[i for i in 1:10]` が新たな配列を生成するため，
不必要なアロケーションを発生させる．

## Machin（マチン）の公式: [MathWorld](https://mathworld.wolfram.com/MachinsFormula.html)
Machinの公式

$$
   4\arctan \dfrac{1}{5} - \arctan \dfrac{1}{239} = \dfrac{\pi}{4}
$$

を用いた円周率の数値計算は収束が速く，実用的である．
$\arctan x$ の原点におけるTaylor展開
$$
 \arctan x = x - \dfrac{x^3}{3} + \dfrac{x^5}{5} - \dfrac{x^7}{7} + \cdots
$$
は $|x| \le 1$ で収束する．
この級数に $x=1/5, 1/239$ を代入すれば，次の級数展開を得られる．

$$
\pi = 16\sum_{i=0}^\infty \frac{(-1)^i}{2i+1} 5^{-(2i+1)}
  - 4\sum_{i=0}^\infty \frac{(-1)^i}{2i+1} 239^{-(2i+1)}
$$

これを適当な項数で打ち切れば円周率の近似値を求めることができる．

#### 練習問題 
Machinの公式で円周率の近似値を計算するプログラムを自分で作成してみよう．
```
    abs(近似値 -  Float64(pi)) == 0.0
```
が `true` になれば `Float64`型に関しては限界近くまで近似できていると判断してください．

## Note: `BigFloat`型で円周率を表示する
任意精度の浮動小数点数を扱うものとして，Juliaでは `BigFloat` 型が用意してある．
`BigFloat`型で `pi` を実体化すれば任意の桁数で 円周率を表示できる．

In [7]:
setprecision(2600)   # 精度を 2600 bit に設定．
big(pi)

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117067982148086513282306647093844609550582231725359408128481117450284102701938521105559644622948954930381964428810975665933446128475648233786783165271201909145648566923460348610454326648213393607260249141273724587006606315588174881520920962829254091715364367892590360011330530548820466521384146951941511609433057270365759591953092186117381932611793105118548074462379962749567351885752724891227938183011949129833673362440656643086021394946395224737190702179860943702770539217176293176752384674818467669405132000568127145263560827785771342757789609173637178721468440901224953430146549585371050792279689258923542019956112129021960864034418159813629774771309960518707211349999998372978049951056

2600 bit は 10進数に換算すると，およそ782桁 ($2600 \log_{10} 2 \approx 782.7$)
に相当する．

In [8]:
2600*log10(2)

782.6779887263511

出力を5桁ごとに区切って整形すると次のようになる．
```
3.
14159 26535 89793 23846 26433 83279 50288 41971 69399 37510   
58209 74944 59230 78164 06286 20899 86280 34825 34211 70679
82148 08651 32823 06647 09384 46095 50582 23172 53594 08128    
48111 74502 84102 70193 85211 05559 64462 29489 54930 38196 
44288 10975 66593 34461 28475 64823 37867 83165 27120 19091
45648 56692 34603 48610 45432 66482 13393 60726 02491 41273
72458 70066 06315 58817 48815 20920 96282 92540 91715 36436 
78925 90360 01133 05305 48820 46652 13841 46951 94151 16094 
33057 27036 57595 91953 09218 61173 81932 61179 31051 18548
07446 23799 62749 56735 18857 52724 89122 79381 83011 94912
98336 73362 44065 66430 86021 39494 63952 24737 19070 21798
60943 70277 05392 17176 29317 67523 84674 81846 76694 05132 
00056 81271 45263 56082 77857 71342 75778 96091 73637 17872 
14684 40901 22495 34301 46549 58537 10507 92279 68925 89235
42019 95611 21290 21960 86403 44181 59813 62977 47713 09960
51870 72113 49999 99837 29780 49951 056
```

## 参考文献
- Petr Beckmann: "A History of $\pi$ ", 1971. （日本語訳：『 $\pi$ の歴史』，ちくま学芸文庫） 
- [MathWorld:Pi](https://mathworld.wolfram.com/Pi.html)
- [MathWorld:PiDigits](https://mathworld.wolfram.com/PiDigits.html)
- OEIS: https://oeis.org/A000796